<a href="https://colab.research.google.com/github/akshit7093/Stock-market-analysis/blob/main/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import pandas as pd
import json


def load_and_preprocess_data(file_path):
    stock_data = pd.read_csv(file_path)
    stock_data['Date'] = pd.to_datetime(stock_data['Date'], errors='coerce')
    stock_data = stock_data.sort_values(by=['Company', 'Date'])
    for col in ['Close/Last', 'Open', 'High', 'Low']:
        stock_data[col] = stock_data[col].str.replace('$', '').astype(float)
    stock_data.rename(columns={
        'Close/Last': 'Close',
        'Open': 'Open',
        'High': 'High',
        'Low': 'Low'
    }, inplace=True)
    return stock_data

# Calculate statistics
def calculate_statistics(stock_data):
    stock_data['Price Change'] = stock_data.groupby('Company')['Close'].diff()
    stock_data['Moving Average'] = stock_data.groupby('Company')['Close'].rolling(window=5).mean().reset_index(level=0, drop=True)
    return stock_data

# Identify trends
def identify_trends(stock_data):
    stock_data['Buy Signal'] = (stock_data['Close'] > stock_data['Moving Average']) & (stock_data['Close'].shift(1) <= stock_data['Moving Average'].shift(1))
    stock_data['Sell Signal'] = (stock_data['Close'] < stock_data['Moving Average']) & (stock_data['Close'].shift(1) >= stock_data['Moving Average'].shift(1))
    return stock_data

# Simulate trades
def simulate_trades(stock_data, initial_cash=10000):
    portfolio = {'cash': initial_cash, 'stocks': {}, 'history': []}

    for date, row in stock_data.iterrows():
        stock = row['Company']

        if row['Buy Signal'] and portfolio['cash'] >= row['Close']:
            portfolio['cash'] -= row['Close']
            if stock not in portfolio['stocks']:
                portfolio['stocks'][stock] = 0
            portfolio['stocks'][stock] += 1
            portfolio['history'].append({
                'date': str(date),
                'company': stock,
                'action': 'BUY',
                'price': row['Close']
            })

        elif row['Sell Signal'] and stock in portfolio['stocks'] and portfolio['stocks'][stock] > 0:
            portfolio['cash'] += row['Close']
            portfolio['stocks'][stock] -= 1
            portfolio['history'].append({
                'date': str(date),
                'company': stock,
                'action': 'SELL',
                'price': row['Close']
            })

    return portfolio

# Generate JSON and text reports
def generate_reports(stock_data, portfolio, json_file_path='portfolio_report.json', txt_file_path='portfolio_report.txt'):
    # Generate JSON report
    report = {
        'trades': portfolio['history'],
        'final_portfolio': {
            'cash': portfolio['cash'],
            'stocks': portfolio['stocks']
        }
    }
    with open(json_file_path, 'w') as json_file:
        json.dump(report, json_file, indent=4)

    # Generate text report
    with open(txt_file_path, 'w') as txt_file:
        txt_file.write("Simulated Trades\n")
        txt_file.write("Date | Company | Action | Price\n")
        for trade in portfolio['history']:
            txt_file.write(f"{trade['date']} | {trade['company']} | {trade['action']} | ${trade['price']:.2f}\n")

        txt_file.write("\nFinal Portfolio Status\n")
        txt_file.write(f"Cash remaining: ${portfolio['cash']:.2f}\n")
        txt_file.write("Stocks:\n")
        for stock, quantity in portfolio['stocks'].items():
            txt_file.write(f"{stock}: {quantity} shares\n")

if __name__ == "__main__":
    # Load and preprocess data
    file_path = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSt2Y9xvGE-fIGK4Bgqgj86QlKVkVr7a-DESPYea7QRYvjFNZuf5w5I59RDKRTbS0eh61Qmq45M-C4X/pub?output=csv'
    stock_data = load_and_preprocess_data(file_path)

    # Calculate statistics
    stock_data = stock_data.groupby('Company', group_keys=False).apply(calculate_statistics)

    # Identify trends
    stock_data = stock_data.groupby('Company', group_keys=False).apply(identify_trends)

    # Simulate trades
    portfolio = simulate_trades(stock_data)

    # Generate reports
    generate_reports(stock_data, portfolio)

    # Display a preview of the stock data and portfolio
    print("Stock Data Preview:")
    print(stock_data.head())
    print("\nPortfolio Summary:")
    print(portfolio)


Stock Data Preview:
     Company       Date    Close       Volume     Open     High      Low  \
2515    AAPL 2013-07-18  15.4199  218632537.0  15.4779  15.5311  15.3789   
2514    AAPL 2013-07-19  15.1768  268548901.0  15.4679  15.4993  15.1554   
2513    AAPL 2013-07-22  15.2254  207648981.0  15.3379  15.3482  15.1953   
2512    AAPL 2013-07-23  14.9639  354477618.0  15.2143  15.2486  14.9539   
2511    AAPL 2013-07-24  15.7325  591624923.0  15.6761  15.8782  15.5450   

      Price Change  Moving Average  Buy Signal  Sell Signal  
2515           NaN             NaN       False        False  
2514       -0.2431             NaN       False        False  
2513        0.0486             NaN       False        False  
2512       -0.2615             NaN       False        False  
2511        0.7686         15.3037       False        False  

Portfolio Summary:
{'cash': 10114.170700000048, 'stocks': {'AAPL': 0, 'AMD': 3, 'AMZN': 1, 'CSCO': 0, 'META': 1, 'MSFT': 0, 'NFLX': 0, 'QCOM': 1, 'SBU